In [1]:
import scqubits as sc
import qutip as qt
import numpy as np
from matplotlib import pyplot as plt
import array_to_latex as a2l
from scipy.linalg import inv

In [2]:
omega = 6.67
g1 = .2

qubit = sc.Transmon(
    EJ = 16.0, 
    EC = 0.2, 
    ng = 0, 
    ncut = 30,
    truncated_dim=10
)

res_a = sc.Oscillator(E_osc=omega, truncated_dim = 20)
res_b = sc.Oscillator(E_osc=omega, truncated_dim = 20)

hs = sc.HilbertSpace([qubit, res_a, res_b])

hs.add_interaction(
    g_strength = g1,
    op1 = qubit.n_operator,
    op2 = res_a.annihilation_operator,
    add_hc = True
)

hs.generate_lookup()

In [28]:
tot_trunc = 40
def trunc(op): #method for truncating Qobjs
    return qt.Qobj(op[:tot_trunc, :tot_trunc])

In [29]:
a = hs.op_in_dressed_eigenbasis(op=res_a.annihilation_operator) #convert subsystem operators to dressed eigenbasis rep
b = hs.op_in_dressed_eigenbasis(op=res_b.annihilation_operator)

In [30]:
#subsystem Hamiltonians
res_a_ham = omega*a.dag()*a
res_b_ham = omega*b.dag()*b
qubit_ham = hs.op_in_dressed_eigenbasis(op=qubit.hamiltonian)

#bare Hamiltonian
bare_ham = trunc(qubit_ham + res_a_ham + res_b_ham)

def state_transformation(state, t): #move to interaction picture
    return (2j*np.pi*bare_ham*t).expm()*state

In [32]:
#calculate dispersive shift
w2 = hs.energy_by_bare_index((1,1,0)) - hs.energy_by_bare_index((1,0,0))
w1 = hs.energy_by_bare_index((0,1,0)) - hs.energy_by_bare_index((0,0,0))
chi = (w2-w1)/2
g = -np.sqrt(3)/2 * chi

0.0

In [33]:
g/(2*np.pi)

0.07957747154594767

In [34]:
bsc = trunc(g/2*(a*b.dag()+b*a.dag())) #beamsplitter Hamiltonian

In [35]:
(evals,) = hs["evals"]
diag_dressed = (
    2*np.pi*qt.Qobj(np.diag(evals),
    dims = [hs.subsystem_dims] * 2)
)

In [36]:
diag_dressed_trunc = trunc(diag_dressed)

In [37]:
def conditional_basis(qubit):
    return [hs.dressed_index((qubit, a,b)) for (a,b) in [(0,0),(0,1),(1,0),(1,1)]]

In [38]:
def prop_col(initial, qubit):
    col = [] 
    tlist = np.linspace(0, 2*np.pi/g, 1000)
    result = qt.sesolve(
            diag_dressed_trunc + 2*np.pi*bsc,
            qt.basis(tot_trunc, initial),
            tlist,
        )
    for idx in conditional_basis(qubit):
        col.append(state_transformation(result.states[-1], tlist[-1])[idx][0][0])
    return col

In [39]:
def conditional_propagator(qubit):
    prop = []
    for idx in conditional_basis(qubit):
        prop.append(prop_col(idx, qubit))
    return prop

In [40]:
Ug = conditional_propagator(0)

In [41]:
Uf = conditional_propagator(2)

In [17]:
a2l.to_ltx(inv(Ug)*Uf)

\begin{bmatrix}
 -0.99 + -0.10j &  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + -0.00j\\
  0.00 + 0.00j & -0.32 + -0.87j & -0.03 + 0.05j &  0.00 + 0.00j\\
  0.00 + 0.00j & -0.07 + 0.01j &  0.64 + 0.56j &  -0.00 + 0.00j\\
  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j &  0.34 + -0.82j
\end{bmatrix}


In [42]:
a2l.to_ltx(np.matrix(Uf))

\begin{bmatrix}
  1.00 + -0.00j &  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.63 + 0.00j &  0.00 + -0.78j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.00 + -0.78j &  0.63 + 0.00j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j & -0.21 + -0.00j
\end{bmatrix}


In [43]:
a2l.to_ltx(np.matrix(Ug))

\begin{bmatrix}
  1.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.63 + -0.00j & -0.00 + -0.78j &  0.00 + 0.00j\\
  0.00 + 0.00j & -0.00 + -0.78j &  0.63 + -0.00j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j & -0.21 + -0.00j
\end{bmatrix}
